
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2021/blob/main/40--spark/04--rdd-sampling.ipynb)

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz -O spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

In [2]:
!pip install -q pyspark findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Spark rdd")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

21/12/11 07:34:16 WARN Utils: Your hostname, Jakubs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.8.8 instead (on interface en0)
21/12/11 07:34:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/11 07:34:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/11 07:34:18 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.


In [4]:
spark

In [5]:
sc = spark.sparkContext

### sample

In [7]:
!mkdir -p var
!wget -c https://s3.eu-central-1.amazonaws.com/jkan.pl/data-science/uek/otomoto.csv -O var/otomoto.csv

--2021-12-11 07:34:55--  https://s3.eu-central-1.amazonaws.com/jkan.pl/data-science/uek/otomoto.csv
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.140.51
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.140.51|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [8]:
raw_data = sc.textFile("var/otomoto.csv")
raw_data.sample?

In [9]:
raw_data_sample = raw_data.sample(False, 0.1, 1234)

In [10]:
type(raw_data_sample), raw_data.count(), raw_data_sample.count()

(pyspark.rdd.PipelinedRDD, 51187, 5023)

In [11]:
raw_data_sample = raw_data.takeSample(False, 1000, 1234)

In [12]:
type(raw_data_sample), len(raw_data_sample)

(list, 1000)

In [13]:
raw_data_sample

['"6039179977";"https://www.otomoto.pl/oferta/audi-a3-1-9-tdi-130-km-doinwestowany-ID6AHMAF.html";"";"Audi";"A3";"Osobowe";"1 900 cm3";"5";"Tak";"8L (1996-2003)";"Kompakt";"2001";"";"";"WAUZZZ8LZ1A082482";"";"";"Tak";"";"Diesel";"";"Francja";"";"Tak";"Używane";"13 październik 2016";"Audi A3 8L";"7200.0";"Manualna";"";"";"3";"Tak";"307 000 km";"";"";"";"Osoby prywatnej";"";"Niebieski";"Na przednie koła";"130 KM"',
 '"6036872112";"https://www.otomoto.pl/oferta/bmw-seria-3-ID6Ay6d2.html";"";"BMW";"Seria 3";"Osobowe";"1 998 cm3";"5";"";"E90 (2005-2012)";"Kombi";"2007";"";"";"";"";"";"";"";"Diesel";"";"";"";"";"Używane";"";"BMW Seria 3 E90";"20500.0";"Manualna";"";"";"5";"Tak";"307 000 km";"";"";"";"Firmy";"318";"Niebieski";"Na tylne koła";"122 KM"',
 '"6039259802";"https://www.otomoto.pl/oferta/hyundai-getz-ID6AI7ma.html";"";"Hyundai";"Getz";"Osobowe";"1 400 cm3";"5";"Tak";"";"Auta małe";"2006";"";"";"KMHBT51DP6U531572";"";"";"";"";"Benzyna+LPG";"";"Polska";"";"";"Używane";"27 czerwiec 200

In [14]:
sc.stop()